# Simple example of determining the largest eigenvalue

This notebook is classical version of the variational quantum aalgorithm <https://en.wikipedia.org/wiki/Variational_quantum_eigensolver> This is the most common algorithm used on quantum computers to compute eigenvalues in quantum chemistry.

* See this blog post for an overview of the method: https://joshuagoings.com/2020/08/20/VQE/
* See also this original paper: O’Malley, Peter JJ, et al. “Scalable quantum simulation of molecular energies.” Physical Review X 6.3 (2016): 031007.  https://journals.aps.org/prx/abstract/10.1103/PhysRevX.6.031007

This notebook doesn't use any qbits, but runs the algorithm for a 2 by 2 matrix.




In [ ]:
import numpy as np
import math
import cmath
from scipy.optimize import minimize

In [ ]:
from numpy import linalg as LA

outfile = "Hstore"

#  Functions to apply Pauli spin matrices to vectors

In [ ]:

def apply_sigma_X(a, cc):
    """
     Pauli X matrix to vector
    """
    a[0,1] += cc    
    a[1,0] += cc    

# Pauli Y matrix
def apply_sigma_Y(a, cc):
   a[0,1] = a[0,1] + complex(0,-cc)
   a[1,0] = a[1,0] + complex(0,cc)

# Pauli Z matrix
def apply_sigma_Z(a, cc):
   a[0,0] = a[0,0] + cc
   a[1,1] = a[1,1] - cc

# Pauli unit matrix
def apply_sigma_unit(a, cc):
   a[0,0] = a[0,0] + cc
   a[1,1] = a[1,1] + cc

# Create a test matrix 

Create a Hermitian matrix parameterized by Pauli spin matricies.

$ H = \sum_{i=1}^3 \sigma_i c_i  $$ +  c_{unit} 1 $

In [ ]:

Hclass = np.zeros((2,2), dtype=complex)

# coefficients of Pauli spin operators
c_x = 0.2 
c_y = 0.4
c_z = 0.6 
c_unit = 5.8

apply_sigma_X(Hclass, c_x)
apply_sigma_Y(Hclass, c_y)
apply_sigma_Z(Hclass, c_z)
apply_sigma_unit(Hclass, c_unit)

print("Hamiltonian = \n" , Hclass)

## Use standard numerical analysis to compute the eigenvalues 

In [ ]:
w, v = LA.eig(Hclass)

print("Classical eigenvalues:")
for xx in w:
    # just print real part
    print(xx.real)

## Variational calculation of eigenvalue
Use the VQE method classically

* A dimension 2 vector is created  depending on two parameters $\theta$ , $\phi$
* See  https://qiskit.org/textbook/ch-applications/vqe-molecules.html#varforms

In [ ]:
#  Check the parameterization.
# https://qiskit.org/textbook/ch-applications/vqe-molecules.html#varforms
#
#  U3 *  (1 0)^t
#
def create_1vec(theta, phi, lamb) :

   vec = np.zeros((2,1), dtype=complex)

   vec[0] = math.cos(theta/2.0) 
   vec[1] = math.sin(theta/2.0) *  cmath.exp( phi*1j)

   return vec


In [ ]:
print("Test parameterization")

ans = create_1vec(1.0, 1.0, 1.0) 

print(ans)

nn = np.vdot(ans,ans)

print("norm = " , nn)



 Eigenvalue estimate 
 $ \lambda =  \frac{ \langle v(\theta,\phi),H v(\theta,\phi) \rangle}  {\langle v(\theta,\phi),v(\theta,\phi) \rangle}  $
 
 where $v(\theta, \phi)$ is a parameterized vector.  
 
Minimum $\lambda$ is found by minimizing with respect to $\theta$ 
and $\phi$.

In [ ]:
#
#  Eigenvalue estimate  (v,H * v) / (v,v)
#  where v is a parameterized vector
#

def  compute_lambda(x) :
    theta = x[0]
    phi = x[1]

    lamb = 0 
    vv = create_1vec(theta, phi, lamb) 

    Mvv = np.matmul(Hclass, vv)

    ans = np.vdot(vv,Mvv) / np.vdot(vv,vv)

    return np.real(ans)


x0 = np.array([1.3, 0.7])
print("First guess of eigenvalue = " , x0)

res = minimize(compute_lambda, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

print("Minimum vector = " , res.x)

vv =  compute_lambda( res.x )

print("vv = " , vv )